## Парсинг текстов
Тексты, которые размещены на сайтах, можно спарсить и сразу записать в нужном нам формате
(!) основная часть текстов существует в виде файлов docx, pdf, txt и обрабатывается в другой тетрадке

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re

## Элоя

In [2]:
url = 'https://stihi.ru/avtor/valenvael&book=3#3'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [14]:
eloya_links = []
base = 'https://stihi.ru/'
for i in soup.find_all('a', {'class':'poemlink'}):
    eloya_links.append(base + i.get('href'))
len(eloya_links)

25

In [26]:
eloya_libretto = {}
for i in tqdm(eloya_links, desc = 'Прогресс парсинга'):
    page = requests.get(i)
    soup = BeautifulSoup(page.text)
    title = soup.find('h1').text.upper()
    text = soup.find('div', {'class':'text'}).text
    eloya_libretto[title] = text
    time.sleep(2)

Прогресс парсинга: 100%|██████████| 25/25 [00:54<00:00,  2.18s/it]


In [29]:
with open('eloya.txt', 'w', encoding = 'utf-8') as file:
    for k, v in eloya_libretto.items():
        file.write(k)
        file.write(v)


## Воздух 2112

In [30]:
url = 'https://stihi.ru/avtor/jasvena&s=0'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [31]:
vozdukh_links = []
base = 'https://stihi.ru/'
for i in soup.find_all('a', {'class':'poemlink'}):
    if 'Воздух 2112' in i.text:
        vozdukh_links.append(base + i.get('href'))
len(vozdukh_links)

13

In [35]:
vozdukh_libretto = {}
for i in tqdm(vozdukh_links, desc = 'Прогресс парсинга'):
    page = requests.get(i)
    soup = BeautifulSoup(page.text)
    title = soup.find('h1').text.upper()
    title = re.sub('- ВОЗДУХ 2112', '', title)
    text = soup.find('div', {'class':'text'}).text
    vozdukh_libretto[title] = text
    time.sleep(2)

Прогресс парсинга:   0%|          | 0/13 [00:00<?, ?it/s]

Прогресс парсинга: 100%|██████████| 13/13 [00:27<00:00,  2.13s/it]


In [39]:
with open('vozdukh.txt', 'w', encoding = 'utf-8') as file:
    count = 0
    for k, v in vozdukh_libretto.items():
        count += 1
        file.write(str(count) + '. ' + k)
        file.write('\n')
        file.write(v)
        file.write('\n\n')

## ИХС Моссовет

In [45]:
url = 'https://jcs-mossoveta.ru/libretto.html'
page = requests.get(url)
page.encoding = 'utf-8'
soup = BeautifulSoup(page.text)

In [ ]:
counter = 0 # счетчик для записывания номера песни

def format_lyrics(lyrics): # пусть питон сразу отформатирует спикеров (указания, кто поет строчки) сразу так, как мне надо
    # ищем формат: (Иисус), в котором они идут
    pattern = re.compile(r'\((.*?)\)')
    # а нам надо в формате ИИСУС: + добавим переход на нов. строку впереди
    formatted_lyrics = pattern.sub(lambda m: '\n' + m.group(1).upper() + ':', lyrics)
    return formatted_lyrics


with open('libretto.txt', 'w', encoding='utf-8') as file:
    
    # все названия  песен
    songs = soup.find_all('h2')
    for song in songs:
        counter +=1
        # название песни
        file.write(str(counter) + '. ' + re.sub('"', '', song.get_text()) + '\n')
        # ищем h3, где лежит текст песни
        lyrics = song.find_next_sibling('h3')
        if lyrics:
            # Форматируем текст песни
            formatted_lyrics = format_lyrics(lyrics.get_text(separator='\n', strip=True))
            # Записываем текст к
            file.write(formatted_lyrics + '\n\n\n')


## Шахматы

In [8]:
url = 'https://chesslamp.ru/libretto.html'
page = requests.get(url)
page.encoding = 'utf-8'
soup = BeautifulSoup(page.text)

In [11]:
songs_raw = soup.find_all('li', {'class':'song'})

In [ ]:
with open('chess_ru.txt', 'w', encoding = 'utf-8') as file:
    counter = 0
    for s in songs_raw:
        title = s.find('h3').text
        text = s.find_all('div', {'class' : 'song__two-columns'})
        if text:
            counter +=1
            song_number = str(counter) + '. '
            file.write(song_number + title.upper() + '\n\n')
            for t in text:
                speaker = t.find('p', {'class' : "song__role"})
                line = t.find('p', {'class' : "song__lyric"})
                if speaker:
                    if re.search(r'[\w]+', speaker.text):
                        file.write(speaker.text.strip().upper() + ':' + '\n')
                if line:
                    to_write = re.sub(r'              ', '', line.text)
                    to_write = re.sub(r'  ', '', to_write)
                    file.write(to_write.strip() + '\n\n')
        file.write('\n') # доп. разрыв для раздления песен
    
    


## Mamma mia

In [2]:
url = 'https://catsinrus.narod.ru/mammamia.htm'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [74]:
def links_extraction(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    linkstosongs = []
    songtitles = []
    link_base = 'https://catsinrus.narod.ru/'
    for i in soup.find_all('a'):
        if 'l.' in i.get('href') and '.htm' in i.get('href'):
            if link_base + i.get('href') not in linkstosongs:
                linkstosongs.append(link_base + i.get('href'))
                title = re.sub(r'[\n\t\r\(\)]', '', i.text)
                if title[-1] == ' ':
                    songtitles.append(title[:-1])
                else:
                    songtitles.append(title)

    libretto = dict(zip(songtitles, linkstosongs))
    return libretto


In [75]:
mammamia_links = links_extraction('https://catsinrus.narod.ru/mammamia.htm')
mammamia_links

{'Under Attack': 'https://catsinrus.narod.ru/l.podognem.htm',
 'Деньги, Деньги, Деньги': 'https://catsinrus.narod.ru/l.money.htm',
 'Кто-то': 'https://catsinrus.narod.ru/l.oneofus.htm',
 'Спасибо тебе за музыку': 'https://catsinrus.narod.ru/l.thank.htm',
 'S. O. S.': 'https://catsinrus.narod.ru/l.sos.htm',
 'Мама мия': 'https://catsinrus.narod.ru/l.mamamia.htm',
 'Если мама будет не прочь': 'https://catsinrus.narod.ru/l.doesyourmother.htm',
 'Чикитита': 'https://catsinrus.narod.ru/l.chikit.htm',
 'Зная всё, зная нас': 'https://catsinrus.narod.ru/l.knowing.htm',
 'Дэнсинг квин': 'https://catsinrus.narod.ru/l.dqueen.htm',
 'Наше последнее лето': 'https://catsinrus.narod.ru/l.ourlast.htm',
 'Дай мне свою любовь': 'https://catsinrus.narod.ru/l.lay.htm',
 'Сквозь пальцы': 'https://catsinrus.narod.ru/l.slipping.htm',
 'Супер Труппа': 'https://catsinrus.narod.ru/l.super.htm',
 'Победителю достаётся всё': 'https://catsinrus.narod.ru/l.thewinner.htm',
 'Дайте! Дайте! Дайте!': 'https://catsinrus

In [132]:
for k, v in mammamia_links.items():
    print (k, v)

Under Attack https://catsinrus.narod.ru/l.podognem.htm
Деньги, Деньги, Деньги https://catsinrus.narod.ru/l.money.htm
Кто-то https://catsinrus.narod.ru/l.oneofus.htm
Спасибо тебе за музыку https://catsinrus.narod.ru/l.thank.htm
S. O. S. https://catsinrus.narod.ru/l.sos.htm
Мама мия https://catsinrus.narod.ru/l.mamamia.htm
Если мама будет не прочь https://catsinrus.narod.ru/l.doesyourmother.htm
Чикитита https://catsinrus.narod.ru/l.chikit.htm
Зная всё, зная нас https://catsinrus.narod.ru/l.knowing.htm
Дэнсинг квин https://catsinrus.narod.ru/l.dqueen.htm
Наше последнее лето https://catsinrus.narod.ru/l.ourlast.htm
Дай мне свою любовь https://catsinrus.narod.ru/l.lay.htm
Сквозь пальцы https://catsinrus.narod.ru/l.slipping.htm
Супер Труппа https://catsinrus.narod.ru/l.super.htm
Победителю достаётся всё https://catsinrus.narod.ru/l.thewinner.htm
Дайте! Дайте! Дайте! https://catsinrus.narod.ru/l.daite.htm
Это шанс https://catsinrus.narod.ru/l.chance.htm
Это что за игра? https://catsinrus.naro

In [151]:
mammamia_links['Это шанс']

'https://catsinrus.narod.ru/l.chance.htm'

In [150]:
with open('mammamia.txt', 'w', encoding='utf-8') as file:
    file.write('MAMMA MIA!\n\n')
    counter = 1
    for k, v in mammamia_links.items():
        page = requests.get(v)
        soup = BeautifulSoup(page.text)

        file.write(str(counter) + '. ' + k.upper() + '\n\n')
        for i in soup.find_all('p')[2:]:
            file.write(re.sub(r'	', '', i.text.strip()))
            file.write('\n')
        file.write('\n\n')
        counter +=1


In [ ]:
for k, v in mammamia_links.items():
    
    page = requests.get(v)
    soup = BeautifulSoup(page)

    with open('mammamia.txt', 'w', encoding='utf-8') as file:
        
        file.write(k.upper())
        for i in soup.find_all('p')[2:]:
            print (i.text)

Не знал я ревности до сих пор, 
Теперь готов я любому парню дать отпор.
Я стал ревнивцем друзьям на смех,
А раньше думал табак мой самый тяжкий грех.
 
Всё сводится к нулю,
Ревную и люблю.
Весь мир вверх дном, 
Я об одном тебя молю.
 
Ты не трать напрасно чувства, просто любовь мне дай.
 
Амура выстрел так меток был.
Лишь парой слов, лишь одной улыбкой меня сгубил.
Он поразил меня глубоко, 
Солидной даме нельзя влюбляться так легко.
Теряю я покой, когда ты не со мной.
И жар в груди, молю приди, мой дорогой!
 
Ты не трать напрасно чувства, просто любовь мне дай.
Без тебя на сердце пусто, просто любовь мне дай.
 


Влюблялась прежде я много раз,
Но тех романов огонь всегда так быстро гас.
 
А я старался разумно жить.
Тебе одной удалось мой разум сокрушить.
 
(                                       ) и явью стали сны.
О как нам быть, ведь мы любить обречены.
 
Ты не трать напрасно чувства, просто любовь мне дай.
Без тебя на сердце пусто, просто любовь мне дай.
Ты не трать напрасно чувства

In [ ]:
page = requests.get(url)
soup = BeautifulSoup(page.text)
links = []
titles = []
libretto = soup.find('table')
for i in libretto.find_all('a'):
    if i.parent.name == 'font':
        links.append(i.get('href'))
        titles.append(i.text)
print (links, titles)

['l.money.htm', 'l.oneofus.htm', 'l.thank.htm', 'l.sos.htm', 'l.doesyourmother.htm', 'l.chikit.htm', 'l.knowing.htm', 'l.dqueen.htm', 'l.ourlast.htm', 'l.lay.htm', 'l.slipping.htm', 'l.super.htm', 'l.thewinner.htm', 'l.daite.htm', 'l.chance.htm', 'l.thename.htm', 'l.dadada.htm', 'l.dadada.htm', 'l.mechta.htm'] ['\nДеньги, Деньги, Деньги (', 'Кто-то ', '\nСпасибо тебе за музыку (', 'S. O. S.', 'Если мама \r\n\t\tбудет не прочь \r\n\t\t', '\nЧикитита (', 'Зная всё, \r\n\t\tзная нас', '\nДэнсинг квин (', 'Наше \r\n\t\tпоследнее лето \r\n\t\t', '\nДай мне свою любовь (', 'Сквозь пальцы', '\nСупер Труппа (', 'Победителю \r\n\t\tдостаётся всё (', '\nДайте! Дайте! Дайте! \n', 'Это шанс', '\nЭто что за игра?', 'Да, Да, Да, \r\n\t\tДа, Да (', 'I \r\n\t\tDo, I Do, I Do, I Do, I Do)', 'Мечта моя']


In [32]:
#titles_clean = re.sub(r'[\n\t\r\(\)]', '', ' '.join(titles))
titles_clean = []
# чистим названия
for t in titles:
    t = re.sub(r'[\n\t\r\(\)]', '', t)
    if t[-1] == ' ': # убираем пробел в конце
        t = t[:-1]
    titles_clean.append(t)
print(titles_clean)

['Деньги, Деньги, Деньги', 'Кто-то', 'Спасибо тебе за музыку', 'S. O. S.', 'Если мама будет не прочь', 'Чикитита', 'Зная всё, зная нас', 'Дэнсинг квин', 'Наше последнее лето', 'Дай мне свою любовь', 'Сквозь пальцы', 'Супер Труппа', 'Победителю достаётся всё', 'Дайте! Дайте! Дайте!', 'Это шанс', 'Это что за игра?', 'Да, Да, Да, Да, Да', 'I Do, I Do, I Do, I Do, I Do', 'Мечта моя']


In [38]:
# сформируем нормальные ссылки
link_base = 'https://catsinrus.narod.ru/'
links_full = [link_base + l for l in links]
print(links_full)


['https://catsinrus.narod.ru/l.money.htm', 'https://catsinrus.narod.ru/l.oneofus.htm', 'https://catsinrus.narod.ru/l.thank.htm', 'https://catsinrus.narod.ru/l.sos.htm', 'https://catsinrus.narod.ru/l.doesyourmother.htm', 'https://catsinrus.narod.ru/l.chikit.htm', 'https://catsinrus.narod.ru/l.knowing.htm', 'https://catsinrus.narod.ru/l.dqueen.htm', 'https://catsinrus.narod.ru/l.ourlast.htm', 'https://catsinrus.narod.ru/l.lay.htm', 'https://catsinrus.narod.ru/l.slipping.htm', 'https://catsinrus.narod.ru/l.super.htm', 'https://catsinrus.narod.ru/l.thewinner.htm', 'https://catsinrus.narod.ru/l.daite.htm', 'https://catsinrus.narod.ru/l.chance.htm', 'https://catsinrus.narod.ru/l.thename.htm', 'https://catsinrus.narod.ru/l.dadada.htm', 'https://catsinrus.narod.ru/l.dadada.htm', 'https://catsinrus.narod.ru/l.mechta.htm']


In [39]:
songs_links = dict(zip(titles_clean, links_full))
songs_links

{'Деньги, Деньги, Деньги': 'https://catsinrus.narod.ru/l.money.htm',
 'Кто-то': 'https://catsinrus.narod.ru/l.oneofus.htm',
 'Спасибо тебе за музыку': 'https://catsinrus.narod.ru/l.thank.htm',
 'S. O. S.': 'https://catsinrus.narod.ru/l.sos.htm',
 'Если мама будет не прочь': 'https://catsinrus.narod.ru/l.doesyourmother.htm',
 'Чикитита': 'https://catsinrus.narod.ru/l.chikit.htm',
 'Зная всё, зная нас': 'https://catsinrus.narod.ru/l.knowing.htm',
 'Дэнсинг квин': 'https://catsinrus.narod.ru/l.dqueen.htm',
 'Наше последнее лето': 'https://catsinrus.narod.ru/l.ourlast.htm',
 'Дай мне свою любовь': 'https://catsinrus.narod.ru/l.lay.htm',
 'Сквозь пальцы': 'https://catsinrus.narod.ru/l.slipping.htm',
 'Супер Труппа': 'https://catsinrus.narod.ru/l.super.htm',
 'Победителю достаётся всё': 'https://catsinrus.narod.ru/l.thewinner.htm',
 'Дайте! Дайте! Дайте!': 'https://catsinrus.narod.ru/l.daite.htm',
 'Это шанс': 'https://catsinrus.narod.ru/l.chance.htm',
 'Это что за игра?': 'https://catsinrus

In [ ]:
# добавляем криво стоявшую на сайте ссылку
songs_links['Под огнем'] = base_link + 'l.podognem.htm'


In [40]:
songs_links

{'Деньги, Деньги, Деньги': 'https://catsinrus.narod.ru/l.money.htm',
 'Кто-то': 'https://catsinrus.narod.ru/l.oneofus.htm',
 'Спасибо тебе за музыку': 'https://catsinrus.narod.ru/l.thank.htm',
 'S. O. S.': 'https://catsinrus.narod.ru/l.sos.htm',
 'Если мама будет не прочь': 'https://catsinrus.narod.ru/l.doesyourmother.htm',
 'Чикитита': 'https://catsinrus.narod.ru/l.chikit.htm',
 'Зная всё, зная нас': 'https://catsinrus.narod.ru/l.knowing.htm',
 'Дэнсинг квин': 'https://catsinrus.narod.ru/l.dqueen.htm',
 'Наше последнее лето': 'https://catsinrus.narod.ru/l.ourlast.htm',
 'Дай мне свою любовь': 'https://catsinrus.narod.ru/l.lay.htm',
 'Сквозь пальцы': 'https://catsinrus.narod.ru/l.slipping.htm',
 'Супер Труппа': 'https://catsinrus.narod.ru/l.super.htm',
 'Победителю достаётся всё': 'https://catsinrus.narod.ru/l.thewinner.htm',
 'Дайте! Дайте! Дайте!': 'https://catsinrus.narod.ru/l.daite.htm',
 'Это шанс': 'https://catsinrus.narod.ru/l.chance.htm',
 'Это что за игра?': 'https://catsinrus

## РиДж

In [ ]:
url = 'https://zvezdi-rj.narod.ru/spekt/libretto.html'
page = requests.get(url)
soup = BeautifulSoup(page.text)
soup

In [174]:
rj_titles = [i.text.strip() for i in soup.find_all('p', {"align" : "center"})[3:]]
len(rj_titles)

49

In [ ]:
# КТО ПОЕТ
#<font align="left" color="#808080" size="4"><i>Все</i></font><font color="#808080"><br/><br/>
#</font>

# ТЕКСТ ПЕСНИ
#<font align="left" color="#808080" size="4">Верона!<br/>Верона!<br/>Верона ждет!</font></p>

# НАЗВАНИЕ ПЕСНИ
#<p align="center">
#<font align="center" color="#C0C0C0" size="4"><b>Ссора на улице</b></font></p>


In [217]:
rj_titles = [i.text.strip() for i in soup.find_all('font', {'align': 'center', 'size' : '5' })]
print(rj_titles)
for i in rj_titles:
    print(i)


['Вступление', 'Верона', 'Вражда', 'Предчувствие любви', 'Сватовство Париса', 'Нужно выйти замуж', 'Короли ночной Вероны', 'Судьба', 'Бал (музыка)', 'Ты и я', 'Продолжение бала', 'Маска Мести', 'Балкон', 'Красавцы и уроды', 'Два крыла любви', 'Благословение', 'Люди говорят', 'Пробил час', 'Дуэль', 'Смерть Меркуцио', 'Кто виноват?', 'Бог, почему?', 'Утро', 'Власть', 'Прощание Ромео и Джульетты', 'Она жена', 'Отец и дочь', 'Ангел в белом', 'Яд', 'Как сказать?', 'Смерть Ромео', 'Смерть Джульетты', 'Господь, прости', 'Плачьте', 'Счастье -Поклоны']
Вступление
Верона
Вражда
Предчувствие любви
Сватовство Париса
Нужно выйти замуж
Короли ночной Вероны
Судьба
Бал (музыка)
Ты и я
Продолжение бала
Маска Мести
Балкон
Красавцы и уроды
Два крыла любви
Благословение
Люди говорят
Пробил час
Дуэль
Смерть Меркуцио
Кто виноват?
Бог, почему?
Утро
Власть
Прощание Ромео и Джульетты
Она жена
Отец и дочь
Ангел в белом
Яд
Как сказать?
Смерть Ромео
Смерть Джульетты
Господь, прости
Плачьте
Счастье -Поклоны


In [226]:
rj_titles_clean = ['Вступление', 'Верона', 'Вражда', 'Предчувствие любви', 'Сватовство Париса', 'Нужно выйти замуж', 'Короли ночной Вероны', 'Судьба', 'Ты и я', 'Маска Мести', 'Балкон', 'Красавцы и уроды', 'Два крыла любви', 'Благословение', 'Люди говорят', 'Пробил час', 'Дуэль', 'Смерть Меркуцио', 'Кто виноват?', 'Бог, почему?', 'Утро', 'Власть', 'Прощание Ромео и Джульетты', 'Она жена', 'Отец и дочь', 'Ангел в белом', 'Яд', 'Как сказать?', 'Смерть Ромео', 'Смерть Джульетты', 'Господь, прости', 'Плачьте', 'Счастье -Поклоны']
print(len(rj_titles_clean), len(rj_titles))

33 35


In [ ]:
content = soup.find_all('tr')[2:]

In [410]:
#3
with open('romeoandjuliette.txt', 'w', encoding = 'utf-8') as file:
    
    for song in content:
        #title = song.find('font', {'align':"center", 'color':"#C0C0C0", 'size':"5"}).text
        #file.write(title + '\n\n')

        text = song.find_all('font', {'align':"left", 'color':"#808080"})
        
        for line in text:
            if line != '':
                speaker_italics = line.find('i')
                if speaker_italics:
                    speaker_italics = speaker_italics.text.strip().upper()
                    if not speaker_italics.endswith(':'):
                        speaker_italics += ':'
                    file.write('\n' + speaker_italics + '\n')
                
                songtext = line.text.strip()
                if speaker_italics:
                    songtext = songtext.replace(speaker_italics, '', 1).strip()
                file.write(songtext + '\n')
        file.write('\n\n')


    
    


In [431]:
clean = []
for i in content:
    text = i.text
    text = re.sub(r'[\t\r]', '', text)
    clean.append(text)
print(clean)

['\n\n\n\n\xa0\n\n\n\xa0\n\n\xa0\n\nВступление\n\nГолос:\n\nВерона – территория войны!\n                   Два клана – Капулетти и Монтекки,\n                   Богатством, родовитостью равны. \n                   Во времена глубокой старины рассорились.\n                   Клубок змеиный их противоречий\n                   Не может быть и речи разобрать\n                   Никто не вспомнит ныне тех причин,\n                   Что послужили поводом раздора.\n                   Покажется грызня собачьей своры\n                   Невинным воркованием голубей,\n                   В сравнении с враждой почтеннейших семей.\n                   Не счесть числа мужчин\n                   Двух этих благороднейших родов,\n                   Что были изувечены, убиты\n                   На поле этой многолетней битвы.\n                   Я это рассказал, чтоб было ясно вам,\n                   Что служит фоном нашего сюжета – \n                   Трагедии «Ромео и Джульетта»\n\xa0\nК списку песе

In [ ]:
# 

with open('romeoandjuliette0.txt', 'w', encoding = 'utf-8') as file:
    counter = 1
    for song in content:
        # находим название
        title = song.find('font', {'align':"center", 'color':"#C0C0C0"})
        if title:
            file.write(str(counter) + '. ' + title.text.strip().upper() + '\n\n')

        text = song.find_all('font', {'align':"left", 'color':"#808080", 'size':"4"})
        
        for line in text:
            if line != '' and line != '\n':
                speaker_italics = line.find('i')

                songtext = line.text.strip()
                songtext = re.sub(r'                  ', '', songtext)
                other_speakers = set(line.find_all('i'))
                

                for i in other_speakers:
                    #if speaker_italics in line:
                        #songtext = songtext.replace(speaker_italics.text, '', 1).strip()
                    if i in line:
                        for t in songtext:
                            
                            if not i.text.endswith(':'):
                                songtext = re.sub(i.text, '\n' + i.text.strip().upper() + ':' + '\n', songtext)
                            else:
                                songtext = re.sub(i.text, '\n' + i.text.strip().upper() + '\n', songtext)
                
                #to_edit = re.findall(r'\b[А-ЯЁ]+\:\s?\n{2}', songtext)
                #for e in to_edit:
                #    songtext.replace(e, e[:-1], 1)

                songtext = re.sub(r'  ', ' ', songtext)
                #songtext = re.sub(r'  ', ' ', songtext)
                

                file.write(songtext.strip() + '\n')

        file.write('\n\n')
        counter +=1


In [504]:
# норм, но повторы спикеров
with open('romeoandjuliette_repet.txt', 'w', encoding = 'utf-8') as file:
    counter = 1
    for song in content[1:]:
        # находим название
        title = song.find('font', {'align':"center", 'color':"#C0C0C0", 'size':'5'})
        if title:
            file.write(str(counter) + '. ' + title.text.strip().upper() + '\n\n')

        text = song.find_all('font', {'align':"left", 'color':"#808080", 'size':"4"})
        
        for line in text:
            if line != '' and line != '\n':
                speaker_italics = line.find('i')
                #if speaker_italics:
                #    to_write_speaker_italics = speaker_italics.text.strip().upper()
                #    if not to_write_speaker_italics.endswith(':'):
                #        to_write_speaker_italics += ':'
                #    file.write('\n' + to_write_speaker_italics + '\n')
                
                songtext = line.text.strip()
                songtext = re.sub(r'                  ', '', songtext)
                other_speakers = set(line.find_all('i'))
                

                for i in other_speakers:
                    if speaker_italics in line:
                        #songtext = songtext.replace(speaker_italics.text, '', 1).strip()
                        for t in songtext:
                            
                            if not i.text.endswith(':'):
                                songtext = re.sub(i.text, '\n' + i.text.strip().upper() + ':' + '\n', songtext)
                            else:
                                songtext = re.sub(i.text, '\n' + i.text.strip().upper() + '\n', songtext)
                
                #to_edit = re.findall(r'\b[А-ЯЁ]+\:\s?\n{2}', songtext)
                #for e in to_edit:
                #    songtext.replace(e, e[:-1], 1)

                songtext = re.sub(r'  ', ' ', songtext)
                songtext = re.sub(r'  ', ' ', songtext)
                songtext = re.sub(r'[\r\t]', '', songtext)
                #songtext = re.sub(r' \n ', '\n', songtext)
                #songtext = re.sub(r'(?<=\s)\n(?=[А-ЯЁ]+\:)', '\n\n', songtext)

                file.write(songtext.strip() + '\n')

        file.write('\n\n')
        counter +=1
